###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2021 Semester 1

## Assignment 1: Pose classification with naive Bayes


**Student ID(s):**     1039169, 1044793


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [1]:
# Load library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
%matplotlib inline

# Create URL
test_csv = "~/pose-classification-with-naive-bayes/COMP30027_2021_assignment1_data/test.csv"
train_csv = "~/pose-classification-with-naive-bayes/COMP30027_2021_assignment1_data/train.csv"
# Load Dataset 
test_df = pd.read_csv(test_csv, header = None)
train_df = pd.read_csv(train_csv, header = None)

# Duplicate Dataset
new_test_df = test_df.copy()
new_train_df = train_df.copy()

In [2]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(df):
    for column in df.columns[1:]:
        # Replace missing value (9999) with median for each column
        df[column] = np.where(df[column] == 9999, np.nan, df[column])
    return df

In [3]:
# Preprocess testing data
new_test_df = preprocess(new_test_df)
new_test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,bridge,126.8358,99.9275,NaN,NaN,47.5551,-9.7848,7.3779,-65.1004,-62.3788,...,-14.5613,NaN,NaN,-30.3392,-41.2163,23.3146,57.5625,-24.6930,61.5094,-34.0565
1,bridge,NaN,NaN,NaN,NaN,NaN,NaN,38.1608,-43.0753,-47.0571,...,NaN,NaN,NaN,NaN,NaN,-1.1496,31.4956,-11.7053,-4.8259,-13.8147
2,bridge,135.8192,99.7615,NaN,NaN,48.2256,-14.1838,16.3069,-131.9711,-153.9583,...,-5.0926,NaN,NaN,-38.7297,-48.6293,50.0013,86.1222,-56.0617,NaN,NaN
3,bridge,-95.5423,-42.9766,-31.4373,-50.1283,NaN,NaN,49.9728,NaN,NaN,...,7.6738,-42.7047,-87.5917,NaN,NaN,44.5574,NaN,NaN,41.6346,NaN
4,bridge,NaN,-3.6378,NaN,NaN,16.4145,-12.7766,NaN,NaN,NaN,...,71.5469,NaN,NaN,2.3956,-73.9424,NaN,NaN,NaN,NaN,NaN


In [4]:
# Preprocess training data 
new_train_df = preprocess(new_train_df)
new_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,bridge,54.8598,33.0166,NaN,NaN,8.9676,-33.5934,-63.2506,NaN,NaN,...,14.5333,NaN,NaN,-30.5872,-57.6515,41.6003,NaN,NaN,NaN,NaN
1,bridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bridge,-87.4883,-38.9467,32.8427,93.5923,NaN,NaN,NaN,NaN,NaN,...,-5.4110,0.091912,-3.8473,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bridge,-140.2714,-107.9045,NaN,NaN,NaN,NaN,-23.9042,65.4627,76.341,...,-25.8142,NaN,NaN,NaN,NaN,28.4441,49.1976,-44.1205,38.9266,-37.7665
4,bridge,171.6807,72.9843,NaN,NaN,-56.6202,-167.9182,-20.1266,NaN,NaN,...,2.1167,NaN,NaN,-38.3855,-28.2247,38.5907,NaN,NaN,NaN,NaN


In [5]:
# Seperate class by feature_train, feature_test, feature_train, feature_test
feature_train = new_train_df.iloc[:,1:]
y_train = new_train_df[0]

feature_test = new_test_df.iloc[:,1:]
y_test = new_test_df[0]

In [6]:
# Find out the prior probability of each pose
target_count = 0
target_dict = {}
target_dict['Pose'] = []
target_dict['Prior'] = []
for target in np.unique(y_train):
    target_dict['Pose'].append(target)
    target_count = sum(y_train == target)
    #target_dict[target] = target_count / len(y_train)
    target_dict['Prior'].append(target_count / len(y_train))

In [7]:
prior_df = pd.DataFrame(target_dict)
prior_df.set_index(['Pose'], inplace = True)
prior_df.head()

,Prior
Pose,
bridge,0.108434
childs,0.092369
downwarddog,0.137885
mountain,0.214190
plank,0.076305


In [8]:
# Group by Mean for each Pose
mean_df = new_train_df.groupby([0]).mean()
mean_df.index.names = ['Pose']
mean_df

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
Pose,,,,,,,,,,,,,,,,,,,,,
bridge,2.177161,12.799641,9.788473,89.918548,0.421379,-35.668633,-22.888805,-59.413168,-2.154617,24.154271,...,5.999911,-22.507896,-39.758087,-25.190003,-44.478301,79.066020,93.007357,-52.181723,50.402328,-175.157880
childs,-4.940768,-3.162657,11.783752,52.291650,-4.209432,-11.313008,-8.853796,-4.826750,-34.663885,9.620484,...,25.153263,-14.584744,-39.682517,-9.313572,-22.248042,35.509215,-6.541216,-10.833731,-9.749197,-12.792773
downwarddog,-10.027591,-8.713010,-24.110220,-13.782303,-22.717928,-41.071859,6.465195,11.840856,15.063516,17.089380,...,40.486705,-15.035385,-72.986083,-20.025099,-64.663750,89.787142,19.233435,-60.686523,34.994317,-76.695133
mountain,-0.007176,-0.108334,-43.016699,-50.895127,43.169189,51.108017,-0.150617,-16.732309,-13.959189,16.623688,...,109.927956,64.608528,29.111031,64.743256,28.939961,9.954995,-72.358276,-159.434010,-72.369811,-159.565578
plank,24.956926,14.121885,9.384726,15.924659,7.493637,5.558243,-9.899770,-25.054482,-41.672226,-8.037731,...,46.367389,-2.959399,-28.068813,-5.414708,-40.421405,26.714821,-5.000711,-47.573506,0.116671,-32.982349
seatedforwardbend,8.070458,2.366815,-42.656051,-30.915376,40.253043,32.823578,-15.089690,-67.182000,-337.610925,95.509927,...,32.321048,-12.618842,-21.851994,-17.628638,-28.099700,14.162823,-8.465962,-39.788525,-8.623364,-15.696050
tree,0.589596,0.745362,-37.293506,-12.247312,38.524275,14.733019,-1.055582,-41.706667,-1.914127,37.214272,...,61.936969,71.716120,99.040862,73.585166,103.289370,-33.665889,-95.502990,-139.170247,-100.622297,-154.245473
trianglepose,-16.164357,-9.247946,-15.924755,-20.158179,0.067926,2.531898,9.236315,-42.636746,-81.635391,68.456147,...,47.080124,31.479872,20.346802,63.405988,78.330719,13.864414,-49.614862,-125.427235,-48.677404,-123.310509
warrior1,-0.443726,-0.522203,-14.092364,-22.033844,19.945515,29.845928,-2.162809,-32.019159,-54.615888,32.657084,...,55.932560,95.143652,125.511697,95.223994,135.161323,-38.286075,-73.368996,-144.124049,-74.980802,-143.239612


In [9]:
# Group by Standard Deviation for each pose
std_df = new_train_df.groupby([0]).std()
std_df.index.names = ['Pose']
std_df

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
Pose,,,,,,,,,,,,,,,,,,,,,
bridge,147.456257,87.261311,69.622367,167.943489,55.129080,70.257168,192.649792,223.998572,170.545546,230.849497,...,51.749262,41.132232,61.945088,40.607552,38.482215,237.044449,284.941234,91.274971,64.481994,627.276181
childs,56.006162,46.003336,60.273060,116.929493,47.316961,93.578294,74.219740,57.038308,72.978024,45.757830,...,15.052119,25.062826,48.386681,11.603254,16.455009,22.390282,22.037552,11.747792,15.423353,8.652542
downwarddog,44.107748,63.947181,80.679930,118.108706,62.840125,102.386779,50.803915,64.698619,104.924389,79.375269,...,33.414806,36.768581,57.893433,24.629972,36.954105,38.530483,42.426542,49.163467,64.924117,63.901549
mountain,2.942956,2.257785,7.989678,18.969064,7.970678,19.072963,1.330744,3.742915,7.174475,3.709626,...,16.102933,15.385883,42.705678,15.185203,42.133322,16.103028,16.110207,17.872827,16.025912,17.942485
plank,111.317950,69.885563,63.634776,75.531466,69.631737,81.256502,42.686509,84.308108,161.405738,96.500655,...,18.698437,52.001937,76.196206,43.809884,72.517996,16.839750,22.497873,36.574533,25.811160,47.192386
seatedforwardbend,80.376668,41.065393,104.504599,153.698631,88.780732,143.063712,96.479943,140.179653,469.681527,166.072925,...,28.670755,39.562791,56.231660,24.684801,54.856356,54.187611,38.391104,23.820199,42.376281,63.037387
tree,9.358049,9.511160,22.982294,23.571910,25.623265,21.275966,5.346032,36.989568,11.275836,33.508775,...,31.123098,36.244469,47.129526,33.964068,47.118968,22.954103,31.843765,64.710685,29.797829,62.476419
trianglepose,72.160950,34.877811,39.585391,53.878611,38.750760,49.831065,56.693266,68.005516,92.486844,64.575275,...,20.740670,88.717138,140.888361,91.984451,140.411577,24.281792,14.550999,20.558093,18.940289,20.589425
warrior1,20.628732,11.557697,35.485022,59.642369,34.804366,55.082014,16.085169,70.339970,107.378093,72.291310,...,23.410431,41.963743,69.866923,34.460070,42.962927,21.753738,27.691372,23.805458,24.917067,23.381064


In [10]:
# x is mean and y is standard deviation
mean_std_df = pd.merge(mean_df, std_df, on = 'Pose', how = 'left')
mean_std_df

,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,...,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y
Pose,,,,,,,,,,,,,,,,,,,,,
bridge,2.177161,12.799641,9.788473,89.918548,0.421379,-35.668633,-22.888805,-59.413168,-2.154617,24.154271,...,51.749262,41.132232,61.945088,40.607552,38.482215,237.044449,284.941234,91.274971,64.481994,627.276181
childs,-4.940768,-3.162657,11.783752,52.291650,-4.209432,-11.313008,-8.853796,-4.826750,-34.663885,9.620484,...,15.052119,25.062826,48.386681,11.603254,16.455009,22.390282,22.037552,11.747792,15.423353,8.652542
downwarddog,-10.027591,-8.713010,-24.110220,-13.782303,-22.717928,-41.071859,6.465195,11.840856,15.063516,17.089380,...,33.414806,36.768581,57.893433,24.629972,36.954105,38.530483,42.426542,49.163467,64.924117,63.901549
mountain,-0.007176,-0.108334,-43.016699,-50.895127,43.169189,51.108017,-0.150617,-16.732309,-13.959189,16.623688,...,16.102933,15.385883,42.705678,15.185203,42.133322,16.103028,16.110207,17.872827,16.025912,17.942485
plank,24.956926,14.121885,9.384726,15.924659,7.493637,5.558243,-9.899770,-25.054482,-41.672226,-8.037731,...,18.698437,52.001937,76.196206,43.809884,72.517996,16.839750,22.497873,36.574533,25.811160,47.192386
seatedforwardbend,8.070458,2.366815,-42.656051,-30.915376,40.253043,32.823578,-15.089690,-67.182000,-337.610925,95.509927,...,28.670755,39.562791,56.231660,24.684801,54.856356,54.187611,38.391104,23.820199,42.376281,63.037387
tree,0.589596,0.745362,-37.293506,-12.247312,38.524275,14.733019,-1.055582,-41.706667,-1.914127,37.214272,...,31.123098,36.244469,47.129526,33.964068,47.118968,22.954103,31.843765,64.710685,29.797829,62.476419
trianglepose,-16.164357,-9.247946,-15.924755,-20.158179,0.067926,2.531898,9.236315,-42.636746,-81.635391,68.456147,...,20.740670,88.717138,140.888361,91.984451,140.411577,24.281792,14.550999,20.558093,18.940289,20.589425
warrior1,-0.443726,-0.522203,-14.092364,-22.033844,19.945515,29.845928,-2.162809,-32.019159,-54.615888,32.657084,...,23.410431,41.963743,69.866923,34.460070,42.962927,21.753738,27.691372,23.805458,24.917067,23.381064


In [11]:
import math
# Gaussian Distribution
x = 2.0
mean = 1.0
std = 1.0
pdf = (1/(std * math.sqrt(2*math.pi))*math.exp(-(1/2) * ((x - mean) / std)**2))
pdf

0.24197072451914337

In [12]:
# Gaussian function
import math
def gaussian_pdf(x, mean, std):
    #if x == np.nan:
        #pdf = 0
    pdf = (1/(std * math.sqrt(2*math.pi))*math.exp(-(1/2) * ((x - mean) / std)**2))
    return pdf

For our training/prediction set, we cannot include the class label when calculating likelihood rather we have to infer the posterior probability of all class labels and take the largest posterior probability -> MAP hypothesis. Link : https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Gaussian_naïve_Bayes

In [13]:
likelihood_dict = {}
likelihood_dict['Pose'] = np.array(new_train_df[0])
y = 0
while y < (len(new_train_df.columns) - 1):
    x = 0
    likelihood_dict['Likelihood_' + str(y+1)] = []
    while x < (len(new_train_df)):
        if np.isnan((new_train_df.iloc[x, y+1])):
            likelihood = np.finfo(float).eps # epsilon for smoothing
        else:
            pose = new_train_df.iloc[x, 0]
            likelihood = gaussian_pdf(new_train_df.iloc[x, y+1], mean_std_df.loc[pose][y], mean_std_df.loc[pose][y+22])

        likelihood_dict['Likelihood_' + str(y+1)].append(likelihood)
        x += 1
    y += 1

In [14]:
likelihood_df = pd.DataFrame(likelihood_dict)
likelihood_df

,Pose,Likelihood_1,Likelihood_2,Likelihood_3,Likelihood_4,Likelihood_5,Likelihood_6,Likelihood_7,Likelihood_8,Likelihood_9,...,Likelihood_13,Likelihood_14,Likelihood_15,Likelihood_16,Likelihood_17,Likelihood_18,Likelihood_19,Likelihood_20,Likelihood_21,Likelihood_22
0,bridge,2.538217e-03,4.450743e-03,2.220446e-16,2.220446e-16,7.150080e-03,5.675838e-03,2.025863e-03,2.220446e-16,2.220446e-16,...,7.605037e-03,2.220446e-16,2.220446e-16,9.737944e-03,9.776968e-03,1.662095e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16
1,bridge,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16
2,bridge,2.248815e-03,3.834666e-03,5.424396e-03,2.374887e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,...,7.523983e-03,8.340148e-03,5.444092e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16
3,bridge,1.696673e-03,1.756319e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.070787e-03,1.524676e-03,2.104113e-03,...,6.381690e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,1.645043e-03,1.383635e-03,4.353761e-03,6.089673e-03,6.209176e-04
4,bridge,1.397367e-03,3.604065e-03,2.220446e-16,2.220446e-16,4.236961e-03,9.656124e-04,2.070603e-03,2.220446e-16,2.220446e-16,...,7.687465e-03,2.220446e-16,2.220446e-16,9.319098e-03,9.482271e-03,1.658629e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16
5,bridge,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.051281e-03,1.638608e-03,2.333330e-03,...,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,1.617642e-03,1.392323e-03,4.040813e-03,6.106652e-03,6.358148e-04
6,bridge,1.785032e-03,2.696035e-03,2.220446e-16,2.220446e-16,4.823927e-03,4.952445e-03,2.034700e-03,1.778493e-03,2.133093e-03,...,6.982122e-03,2.220446e-16,2.220446e-16,9.747831e-03,1.000073e-02,1.644542e-03,1.388411e-03,4.199646e-03,6.155588e-03,6.197768e-04
7,bridge,2.220446e-16,3.439011e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,2.021228e-03,2.220446e-16,2.220446e-16,...,1.018018e-03,2.220446e-16,2.220446e-16,2.220446e-16,2.220446e-16,1.661802e-03,2.220446e-16,2.220446e-16,2.474113e-03,6.330314e-04
8,bridge,1.708096e-03,2.460602e-03,2.220446e-16,2.220446e-16,3.789507e-03,5.215817e-03,2.022269e-03,1.779800e-03,1.979593e-03,...,7.151193e-03,2.220446e-16,2.220446e-16,9.294374e-03,1.034605e-02,1.643954e-03,1.394453e-03,4.169455e-03,6.006983e-03,6.206758e-04
9,bridge,2.220446e-16,2.794690e-03,2.220446e-16,2.220446e-16,8.401927e-04,3.307165e-04,2.056414e-03,1.775662e-03,2.057862e-03,...,2.001105e-03,2.220446e-16,2.220446e-16,3.197869e-03,1.029489e-02,1.675842e-03,1.340630e-03,4.120082e-03,4.981558e-03,6.336342e-04


In [15]:
posterior_dict = {}
posterior_dict['Pose'] = np.array(new_train_df[0])
i = 0
posterior_list = []
while i < len(likelihood_df):
    j = 1
    likelihood_sum = 0
    while j < len(likelihood_df.columns):
        likelihood_sum += math.log(likelihood_df.iloc[i, j])
        j += 1
    curr_pose = likelihood_df.iloc[i, 0]
    curr_prior = prior_df.loc[curr_pose]
    posterior = math.log(curr_prior) + likelihood_sum
    posterior_list.append(posterior)
    i += 1
posterior_dict['Posterior'] = posterior_list
posterior_df = pd.DataFrame(posterior_dict)
posterior_df

,Pose,Posterior
0,bridge,-487.702514
1,bridge,-795.181991
2,bridge,-549.009703
3,bridge,-375.632079
4,bridge,-490.701854
5,bridge,-497.460171
6,bridge,-250.682315
7,bridge,-558.416990
8,bridge,-251.103505
9,bridge,-319.096321


In [19]:
# maximum a posteriori (MAP) hypothesis
# link : https://machinelearningmastery.com/naive-bayes-for-machine-learning/
# link : https://machinelearningmastery.com/maximum-a-posteriori-estimation/
posterior_df.groupby(['Pose']).max() 
#A popular replacement for maximizing the likelihood is maximizing the 
#Bayesian posterior probability density of the parameters instead

# Helpful link : https://machinelearningmastery.com/classification-as-conditional-probability-and-the-naive-bayes-algorithm/

,Posterior
Pose,
bridge,-129.431406
childs,-179.246730
downwarddog,-115.575723
mountain,-74.825335
plank,-115.959894
seatedforwardbend,-164.682637
tree,-97.028114
trianglepose,-110.952748
warrior1,-105.997459


In [16]:
# This function should calculate prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train():
    return

In [17]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict():
    return

In [18]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate():
    return

## Questions 


If you are in a group of 1, you will respond to **two** questions of your choosing.

If you are in a group of 2, you will respond to **four** questions of your choosing.

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer should be submitted separately as a PDF.

### Q1
Since this is a multiclass classification problem, there are multiple ways to compute precision, recall, and F-score for this classifier. Implement at least two of the methods from the "Model Evaluation" lecture and discuss any differences between them. (The implementation should be your own and should not just call a pre-existing function.)

### Q2
The Gaussian naıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in this dataset? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the classifier’s predictions.

### Q3
Implement a kernel density estimate (KDE) naive Bayes classifier and compare its performance to the Gaussian naive Bayes classifier. Recall that KDE has kernel bandwidth as a free parameter -- you can choose an arbitrary value for this, but a value in the range 5-25 is recommended. Discuss any differences you observe between the Gaussian and KDE naive Bayes classifiers. (As with the Gaussian naive Bayes, this KDE naive Bayes implementation should be your own and should not just call a pre-existing function.)

### Q4
Instead of using an arbitrary kernel bandwidth for the KDE naive Bayes classifier, use random hold-out or cross-validation to choose the kernel bandwidth. Discuss how this changes the model performance compared to using an arbitrary kernel bandwidth.

### Q5
Naive Bayes ignores missing values, but in pose recognition tasks the missing values can be informative. Missing values indicate that some part of the body was obscured and sometimes this is relevant to the pose (e.g., holding one hand behind the back). Are missing values useful for this task? Implement a method that incorporates information about missing values and demonstrate whether it changes the classification results.

### Q6
Engineer your own pose features from the provided keypoints. Instead of using the (x,y) positions of keypoints, you might consider the angles of the limbs or body, or the distances between pairs of keypoints. How does a naive Bayes classifier based on your engineered features compare to the classifier using (x,y) values? Please note that we are interested in explainable features for pose recognition, so simply putting the (x,y) values in a neural network or similar to get an arbitrary embedding will not receive full credit for this question. You should be able to explain the rationale behind your proposed features. Also, don't forget the conditional independence assumption of naive Bayes when proposing new features -- a large set of highly-correlated features may not work well.